Write a program to demonstrate the working of the decision tree based ID3
algorithm. Use an appropriate data set for building the decision tree and
apply this knowledge to classify a new sample. 


In [1]:
import pandas as pd
import numpy as np
from math import log2


In [2]:
data = {
    'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 'Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain', 'Sunny', 'Overcast', 'Overcast', 'Rain'],
    'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild'],
    'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High'],
    'Wind': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Strong'],
    'PlayTennis': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']
}

df = pd.DataFrame(data)
print(df)


     Outlook Temperature Humidity    Wind PlayTennis
0      Sunny         Hot     High    Weak         No
1      Sunny         Hot     High  Strong         No
2   Overcast         Hot     High    Weak        Yes
3       Rain        Mild     High    Weak        Yes
4       Rain        Cool   Normal    Weak        Yes
5       Rain        Cool   Normal  Strong         No
6   Overcast        Cool   Normal  Strong        Yes
7      Sunny        Mild     High    Weak         No
8      Sunny        Cool   Normal    Weak        Yes
9       Rain        Mild   Normal    Weak        Yes
10     Sunny        Mild   Normal  Strong        Yes
11  Overcast        Mild     High  Strong        Yes
12  Overcast         Hot   Normal    Weak        Yes
13      Rain        Mild     High  Strong         No


In [3]:
# Calculate Entropy
def entropy(target_col):
    elements, counts = np.unique(target_col, return_counts=True)
    entropy_value = 0
    for i in range(len(elements)):
        probability = counts[i]/np.sum(counts)
        entropy_value += -probability * np.log2(probability)
    return entropy_value


# Calculate Information Gain
def info_gain(data, split_attribute_name, target_name="PlayTennis"):
    total_entropy = entropy(data[target_name])
    values, counts = np.unique(data[split_attribute_name], return_counts=True)
    weighted_entropy = 0
    for i in range(len(values)):
        subset = data.where(data[split_attribute_name] == values[i]).dropna()
        weighted_entropy += (counts[i]/np.sum(counts)) * entropy(subset[target_name])
    information_gain = total_entropy - weighted_entropy
    return information_gain


In [4]:
def id3(data, originaldata, features, target_attribute_name="PlayTennis", parent_node_class=None):
    # If all target values have the same class → return that class
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    
    # If dataset is empty → return most frequent class from original data
    elif len(data) == 0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name], return_counts=True)[1])]
    
    # If no more features → return parent node class
    elif len(features) == 0:
        return parent_node_class
    
    # Otherwise, build tree recursively
    else:
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name], return_counts=True)[1])]
        
        # Compute Information Gain for each feature
        item_values = [info_gain(data, feature, target_attribute_name) for feature in features]
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        
        # Create tree structure
        tree = {best_feature: {}}
        features = [f for f in features if f != best_feature]
        
        for value in np.unique(data[best_feature]):
            sub_data = data.where(data[best_feature] == value).dropna()
            subtree = id3(sub_data, data, features, target_attribute_name, parent_node_class)
            tree[best_feature][value] = subtree
        
        return tree


In [5]:
features = list(df.columns[:-1])  # All columns except target
target = "PlayTennis"

tree = id3(df, df, features, target)
print("\nGenerated Decision Tree (ID3):\n")
print(tree)



Generated Decision Tree (ID3):

{'Outlook': {'Overcast': 'Yes', 'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}, 'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}


In [6]:
def predict(query, tree, default="Yes"):
    for key in list(query.keys()):
        if key in tree:
            try:
                result = tree[key][query[key]]
            except:
                return default
            
            if isinstance(result, dict):
                return predict(query, result)
            else:
                return result


In [7]:
query = {'Outlook': 'Sunny', 'Temperature': 'Cool', 'Humidity': 'Normal', 'Wind': 'Strong'}
prediction = predict(query, tree)
print("\nPrediction for new sample:", query)
print("Result →", prediction)



Prediction for new sample: {'Outlook': 'Sunny', 'Temperature': 'Cool', 'Humidity': 'Normal', 'Wind': 'Strong'}
Result → Yes


In [8]:
queries = df.iloc[:, :-1].to_dict(orient="records")

predictions = []
for q in queries:
    predictions.append(predict(q, tree))

print("\nPredictions on Training Data:")
print(predictions)

accuracy = np.mean(predictions == df["PlayTennis"])
print("\nTraining Accuracy:", round(accuracy * 100, 2), "%")



Predictions on Training Data:
['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']

Training Accuracy: 100.0 %


Generated Decision Tree (ID3):
{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}

Prediction for new sample: {'Outlook': 'Sunny', 'Temperature': 'Cool', 'Humidity': 'Normal', 'Wind': 'Strong'}
Result → Yes
